In [2]:
#MeanShift Tracking
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

ret, frame = cap.read()
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rectangles = face_cascade.detectMultiScale(frame)

(face_x,face_y,w,h) = tuple(face_rectangles[0])
track_window = (face_x,face_y,w,h)

#setup roi for tracking
roi = frame[face_y:face_y+h,face_x:face_x+w]

hsv_roi =  cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_histogram = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_histogram,roi_histogram,0,255,cv2.NORM_MINMAX)

#set up termination criteria
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret, frame = cap.read()
    if ret == True: 
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_histogram,[0,180],1)
        
        #update and apply meanshift
        ret, track_window = cv2.meanShift(dst, track_window, term_criteria)
        x,y,w,h = track_window
        image = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
        
        cv2.imshow('Tracking',image)
        
        k=cv2.waitKey(30) & 0xFF
        if k == 27:
            break
    else:
        break
        
cv2.destroyAllWindows()
cap.release()
 

In [6]:
#CamShift

import cv2
import numpy as np

cap = cv2.VideoCapture(0)

ret, frame = cap.read()
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rectangles = face_cascade.detectMultiScale(frame)

(face_x,face_y,w,h) = tuple(face_rectangles[0])
track_window = (face_x,face_y,w,h)

#setup roi for tracking
roi = frame[face_y:face_y+h,face_x:face_x+w]

hsv_roi =  cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_histogram = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_histogram,roi_histogram,0,255,cv2.NORM_MINMAX)

#set up termination criteria
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret, frame = cap.read()
    if ret == True: 
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_histogram,[0,180],1)
        
        
        #update and apply CamShift
        ret, track_window = cv2.CamShift(dst, track_window, term_criteria)
        
        #draw on the image based off the points
        points = cv2.boxPoints(ret)
        #convert to integers
        points = np.int0(points)
        
        image = cv2.polylines(frame,[points],True,(0,0,255),5)
        
        cv2.imshow('Tracking',image)
        
        k=cv2.waitKey(30) & 0xFF
        if k == 27:
            break
    else:
        break
        
cv2.destroyAllWindows()
cap.release()
 